# ETL-Yelp!

### Librerías

In [1]:
import pandas as pd
# Se configura pandas para que se muestre la totalidad de las columnas del dataframe.
pd.options.display.max_columns = None
# Se configura pandas para que muestre los valores float en formato de dos decimales.
#pd.options.display.float_format = '{:0.2f}'.format
import os
import ast
import json
import pyarrow as pa
import pyarrow.parquet as pq
# Se importa la función "Nominatim" de la librería "geopy" para utilizarse en la corrección de los datos geográficos.
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut, GeocoderServiceError
import time
import warnings
warnings.filterwarnings("ignore")


### Rutas a los archivos.

In [2]:
path_buisness = 'C:\\Users\\fedez\\OneDrive\\Escritorio\\ProyectoG4-Google_Yelp\\Data\\Raw\\business.pkl'
path_checkin = 'C:\\Users\\fedez\\OneDrive\\Escritorio\\ProyectoG4-Google_Yelp\\Data\\Raw\\checkin.json'
path_review = 'C:\\Users\\fedez\\OneDrive\\Escritorio\\ProyectoG4-Google_Yelp\\Data\\Raw\\review.json'
path_tip = 'C:\\Users\\fedez\\OneDrive\\Escritorio\\ProyectoG4-Google_Yelp\\Data\\Raw\\tip.json'
path_user = 'C:\\Users\\fedez\\OneDrive\\Escritorio\\ProyectoG4-Google_Yelp\\Data\\Raw\\user.parquet'

## Archivo "business.pkl"

### Extracción

In [3]:
df_business = pd.read_pickle(path_buisness)

In [5]:
df_business.head(5)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
Index: 150346 entries, 0 to 150345
Data columns (total 28 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   business_id   150346 non-null  object
 1   name          150346 non-null  object
 2   address       150346 non-null  object
 3   city          150346 non-null  object
 4   state         150343 non-null  object
 5   postal_code   150346 non-null  object
 6   latitude      150346 non-null  object
 7   longitude     150346 non-null  object
 8   stars         150346 non-null  object
 9   review_count  150346 non-null  object
 10  is_open       150346 non-null  object
 11  attributes    136602 non-null  object
 12  categories    150243 non-null  object
 13  hours         127123 non-null  object
 14  business_id   5 non-null       object
 15  name          5 non-null       object
 16  address       5 non-null       object
 17  city          5 non-null       object
 18  state         5 non-null     

### Valores nulos.

In [6]:
df_business.isna().sum()

business_id          0
name                 0
address              0
city                 0
state                3
postal_code          0
latitude             0
longitude            0
stars                0
review_count         0
is_open              0
attributes       13744
categories         103
hours            23223
business_id     150341
name            150341
address         150341
city            150341
state           150341
postal_code     150341
latitude        150341
longitude       150341
stars           150341
review_count    150341
is_open         150341
attributes      150341
categories      150341
hours           150341
dtype: int64

Al observar la existencia de columnas duplicadas cuyo contenido está compuesto en su mayor parte de valores nulos se procede a eliminarlas.

In [7]:
df_business = df_business.loc[:, ~df_business.columns.duplicated()]

Se corrobora que se hayan efectuado los cambios.

In [8]:
df_business.isnull().sum()

business_id         0
name                0
address             0
city                0
state               3
postal_code         0
latitude            0
longitude           0
stars               0
review_count        0
is_open             0
attributes      13744
categories        103
hours           23223
dtype: int64

Se puede distinguir que las columnas "attributes", "categories" y " hours" son las únicas que poseen valores nulos.

### Manejo de columnas.

### Columna "business_id".

In [9]:
duplicados_business = df_business['business_id'].duplicated()
df_duplicados_business = df_business[duplicados_business]
df_duplicados_business.shape

(0, 14)

La columna "business_id" no presenta valores duplicados.

### Columnas "categories"

In [10]:
categories_lists = df_business['categories'].str.split(', ')

# Apilar todas las listas resultantes en una sola lista
all_categories = [category for sublist in categories_lists if sublist is not None for category in sublist]

# Obtener los valores únicos de la lista
unique_categories = set(all_categories)
print(f'La cantidad de categorías únicas es {len(unique_categories)}')

La cantidad de categorías únicas es 1311


Se crea el dataframe "df_restaurants" en base a las filas que contienen el valor "restaurant" y "mex" dentro de ellas con el objetivo de reducir la cantidad de filas a solo las que serán utilizadas.

In [11]:
filtro_mex_restaurant = df_business['categories'].str.contains('(?=.*mex)(?=.*restaurant)', case=False, na=False)
df_restaurants = df_business[filtro_mex_restaurant]
df_restaurants = df_restaurants.reset_index(drop=True)

In [12]:
df_restaurants.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4768 entries, 0 to 4767
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   business_id   4768 non-null   object
 1   name          4768 non-null   object
 2   address       4768 non-null   object
 3   city          4768 non-null   object
 4   state         4768 non-null   object
 5   postal_code   4768 non-null   object
 6   latitude      4768 non-null   object
 7   longitude     4768 non-null   object
 8   stars         4768 non-null   object
 9   review_count  4768 non-null   object
 10  is_open       4768 non-null   object
 11  attributes    4689 non-null   object
 12  categories    4768 non-null   object
 13  hours         4073 non-null   object
dtypes: object(14)
memory usage: 521.6+ KB


In [13]:
top_rest = df_restaurants['name'].value_counts().head(20)
print(top_rest)

name
Taco Bell                        363
Chipotle Mexican Grill           156
Jack in the Box                   96
QDOBA Mexican Eats                81
Chili's                           80
Moe's Southwest Grill             50
Tijuana Flats                     25
Taco Bus                          14
Del Taco                          14
Los Betos                         13
Las Palmas Mexican Restaurant     12
California Tortilla               10
Baja Fresh                        10
Mucho Burrito                      9
Blue Coast Burrito                 9
La Hacienda                        9
Pancheros Mexican Grill            9
El Pollo Loco                      9
Taco Del Mar                       9
El Rodeo                           8
Name: count, dtype: int64


### Columna "review_count".  
Verificación de outliers.

Se convierten los valores de la columna a numeros enteros.

In [14]:
df_restaurants['review_count'] = df_restaurants['review_count'].astype(int)

In [15]:
df_restaurants['review_count'].describe()


count    4768.000000
mean       90.294463
std       167.613747
min         5.000000
25%        16.000000
50%        39.000000
75%        93.250000
max      3834.000000
Name: review_count, dtype: float64

Se observa que la cantidad mínima de reseñas para un negocio son 5 y la máxima 3834, manteniendo un promedio de casi 90 reseñas por negocio.

In [16]:
# Se renombra la columna a "restaurant review count".
df_restaurants.rename(columns={'review_count': 'restaurant review count'}, inplace=True)

### Columna "stars".

Verificación de outliers.

Se convierten los valores de la columna "stars" al tipo float.

In [17]:
df_restaurants['stars'] = df_restaurants['stars'].astype(float)

In [18]:
description = df_restaurants['stars'].describe()
print(description)

count    4768.000000
mean        3.500734
std         0.865904
min         1.000000
25%         3.000000
50%         3.500000
75%         4.000000
max         5.000000
Name: stars, dtype: float64


Se aprecia que la columna no presenta valores atípicos.

In [19]:
# Se renombra la columna a "stars business".
df_business.rename(columns={'stars': 'stars business'}, inplace=True)

### Columna "city".

In [22]:
df_filtro = df_restaurants[df_restaurants['state'] == "FL"]
ciudades_unicas_FL = df_filtro['city'].unique()
ciudades_unicas_FL

array(['New Orleans', 'Hermitage', 'Odessa', 'Horsham', 'Sparks', 'Media',
       'Tucson', 'Pinellas Park', 'Indianapolis', 'Nashville',
       'Collinsville', 'Gilbertsville', 'Wimauma', 'Goleta',
       'Philadelphia', 'Springfield', 'Smyrna', 'Norristown',
       'Warminster', 'Woodbury', 'Reno', 'Ardmore', 'Edmonton', 'Boise',
       'Gretna', 'Tampa', 'Marlton', 'Kenner', 'Largo', 'Doylestown',
       'Feasterville-Trevose', 'Oldsmar', 'Wilmington', 'New Hope',
       'Clearwater', 'Ruskin', 'Mount Holly', 'Saint Petersburg',
       'Oro Valley', 'Palm Harbor', 'Dover', 'Saint Louis', 'Brandon',
       'Lutz', "Land O' Lakes", 'Carmel', 'Marana', 'North Wales',
       'New Castle', 'Bala Cynwyd', 'Belleville', 'Pinecrest West Park',
       'St. Louis', 'Florissant', 'Belle Chasse', 'New Port Richey',
       'Zionsville', 'St. Petersburg', 'Bridgeton', 'Exton', 'Plainfield',
       'Hendersonville', 'Marlborough', 'Fishers', 'Willow Grove',
       'Fenton', 'Harahan', 'Eagleville'

Se identificó que varias de las ciudades que figuran en el dataset no coinciden con las coordenadas correspondientes por fila, por lo que se procede a cambiar el nombre de la ciudad por el que realmente corresponde por geolocación.

In [23]:
# Se inicializa el geolocalizador.
geolocator = Nominatim(user_agent="myGeocoder")

# Se crea la función para rastrear la ciudad correspondiente a las coorenadas.
def get_city(lat, lon, retries=3):

    for attempt in range(retries):
        try:
            location = geolocator.reverse((lat, lon), language='en')
            address = location.raw.get('address', {})
            city = address.get('city') or address.get('town') or address.get('village') or address.get('hamlet')
            return city if city else 'SD'
        
        except (GeocoderTimedOut, GeocoderServiceError):
            if attempt < retries - 1:
                # Se espera un segundo antes de reintentar.
                time.sleep(1)  
                continue
            # En caso de no lograr obtener el nombre de la ciudad se retorna la sigla "SD"(Sin Datos)
            return 'SD'

# Se aplica la función para modificar los nombres de la columna.
df_restaurants['city'] = df_restaurants.apply(lambda row: get_city(row['latitude'], row['longitude']), axis=1)

In [24]:
top_city = df_restaurants['city'].value_counts().head(10)

print(top_city)

city
Tucson                430
Indianapolis          382
Nashville-Davidson    346
Philadelphia          325
Reno                  195
Tampa                 186
SD                    167
St. Louis             133
New Orleans           125
Santa Barbara         117
Name: count, dtype: int64


De las ciudades que fueron revisadas por la ubicación geográfica hubo 167 que no pudieron ser rastreadas por lo que se procedió a imputar los valores faltantes por la sigla "SD"(Sin Datos).

### Columna "state".

Así como se sucedió anteriormente con la columna "city", los elementos de la columna "state" también se vieron afectados por la inconsistencia entre el nombre y las coordenadas, por lo que se preocede a cambiar el nombre del estado por el que corresponde por la geolocación.

In [26]:
geolocator = Nominatim(user_agent="myGeocoder")

# Se crea la función para rastrear el estado correspondiente a las coorenadas.
def get_state(lat, lon):
    try:
        location = geolocator.reverse((lat, lon), language='en')
        address = location.raw['address']
        return address.get('state', '')
    except (GeocoderTimedOut, GeocoderServiceError):
        return None
    
# Se aplica la función para modificar los nombres de la columna.
df_restaurants['state'] = df_restaurants.apply(lambda row: get_state(row['latitude'], row['longitude']), axis=1)

In [28]:
top = df_restaurants['state'].value_counts().head(14)

print(top)

state
Pennsylvania    732
Florida         709
Arizona         550
Tennessee       523
Indiana         511
Missouri        364
Nevada          277
Louisiana       237
California      192
New Jersey      189
Idaho           155
Illinois        108
Delaware         89
Alberta          62
Name: count, dtype: int64


Siendo Florida el estado objetivo del proyecto por su gran poblacion de origen latino y concentración de restaurantes mexicanos se opta por filtrar las filas que correspondan a ese estado.

In [34]:
filtro_florida = df_restaurants['state'].str.contains('Florida', case=False, na=False)
df_florida = df_restaurants[filtro_florida]
df_business = df_florida.reset_index(drop=True)

In [35]:
df_business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709 entries, 0 to 708
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   business_id              709 non-null    object 
 1   name                     709 non-null    object 
 2   address                  709 non-null    object 
 3   city                     709 non-null    object 
 4   state                    709 non-null    object 
 5   postal_code              709 non-null    object 
 6   latitude                 709 non-null    object 
 7   longitude                709 non-null    object 
 8   stars                    709 non-null    float64
 9   restaurant review count  709 non-null    int32  
 10  is_open                  709 non-null    object 
 11  attributes               698 non-null    object 
 12  categories               709 non-null    object 
 13  hours                    638 non-null    object 
dtypes: float64(1), int32(1), o

### Columna "hours".

Se desanidan los valores de la columna "hours" y se crea una columna específica para cada día de la semana.

In [36]:
df_hours = pd.json_normalize(df_business['hours'])

# Se concatenan los datos desanidados al dataFrame original y se elimina la columna "hours".
df_horarios = pd.concat([df_business.drop(columns=['hours']), df_hours], axis=1)


Se renombran los nombres de las columnas para tener una idea más acertada del contenido de las columnas.

In [ ]:
nuevo_nombre = {
    'Monday': 'Lunes horarios',
    'Tuesday': 'Martes horarios',
    'Wednesday': 'Miércoles horarios',
    'Thursday': 'Jueves horarios',
    'Friday': 'Viernes horarios',
    'Saturday': 'Sábado horarios',
    'Sunday': 'Domingo horarios'
}

# Renombrar las columnas
df_business = df_horarios.rename(columns=nuevo_nombre, inplace=True)

# Se reordenan las columnas según el día de la semana.
columnas_orden = ['Lunes horarios', 'Martes horarios', 'Miércoles horarios', 'Jueves horarios', 'Viernes horarios', 'Sábado horarios', 'Domingo horarios']
df_business = df_business[columnas_orden]

In [43]:
df_business = df_horarios

### Eliminación de columnas.

Se opta por eliminar las columnas "address", "postal_code", "is_open" y "attributes" debido a que no aporta datos relevantes.

In [44]:
df_business_final = df_business.drop(['address','postal_code', 'is_open', 'attributes'], axis=1)

In [50]:
df_business_final.head(3)

,business_id,name,city,state,latitude,longitude,stars business,restaurant review count,categories,Lunes horarios,Martes horarios,Miércoles horarios,Jueves horarios,Viernes horarios,Sábado horarios,Domingo horarios
0,aNtKyc2rr-uK5cqzY9TVQQ,Chipotle Mexican Grill,Largo,Florida,27.894167,-82.779866,3.0,19,"Mexican, Fast Food, Restaurants",0:0-0:0,10:45-22:0,10:45-22:0,10:45-22:0,10:45-22:0,10:45-22:0,10:45-22:0
1,XwoXi0M66RU6Gouq-DSDHw,G Peppers Grill & Tavern,Citrus Park,Florida,28.092136,-82.578574,4.0,82,"Restaurants, Mexican, American (Traditional)",11:0-22:0,11:0-22:0,11:0-22:0,11:0-22:0,11:0-23:0,11:0-23:0,10:30-21:0
2,uKEabBQrn0gLzvTppOrKIA,Casita Taqueria,Saint Petersburg,Florida,27.796976,-82.638168,4.5,207,"Mexican, Restaurants",11:0-21:0,11:0-21:0,11:0-21:0,11:0-21:0,11:0-22:0,11:0-22:0,NaN


In [51]:
df_business_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709 entries, 0 to 708
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   business_id              709 non-null    object 
 1   name                     709 non-null    object 
 2   city                     709 non-null    object 
 3   state                    709 non-null    object 
 4   latitude                 709 non-null    object 
 5   longitude                709 non-null    object 
 6   stars business           709 non-null    float64
 7   restaurant review count  709 non-null    int32  
 8   categories               709 non-null    object 
 9   Lunes horarios           585 non-null    object 
 10  Martes horarios          619 non-null    object 
 11  Miércoles horarios       620 non-null    object 
 12  Jueves horarios          630 non-null    object 
 13  Viernes horarios         631 non-null    object 
 14  Sábado horarios          6

### Al haberse realizado una reducción de los datos a través del dataframe "df_florida" se usará los datos de la columna "business_id" para descartar las filas de los demás datasets que no coincidan con dicho ID.

In [3]:
business_ids = df_business_final['business_id'].unique()
print(f"El filtro consta de {len(business_ids)} ID's únicos")

El filtro consta de 709 ID's únicos


## Archivo "review.json"

### Extracción

Se extraen los datos del archivo "review.json" para pasarlo a formato parquet.

In [ ]:
path_json = "C:\\Users\\fedez\\OneDrive\\Escritorio\\ProyectoG4-Google_Yelp\\Data\\Raw\\review.json"
path_review_parquet = "C:\\Users\\fedez\\OneDrive\\Escritorio\\ProyectoG4-Google_Yelp\\Data\\Raw\\review.parquet"

# Se lee el archivo json en fragmentos para manejar archivos grandes
chunksize = 100000  # Número de líneas por fragmento
chunks = pd.read_json(path_review, lines=True, chunksize=chunksize)

# Se crea una lista para almacenar las tablas PyArrow
tables = []

# Se procesa y convierte cada fragmento en una tabla PyArrow
for chunk in chunks:
    table = pa.Table.from_pandas(chunk)
    tables.append(table)

# Se concatenan todas las tablas en una sola única tabla.
final_table = pa.concat_tables(tables)

# Guardar la tabla final en un archivo Parquet
pq.write_table(final_table, path_review_parquet)

In [ ]:
# Se abre el archivo
parquet_file = pq.ParquetFile(path_review_parquet)

# Se obtiene el número total de grupos de filas.
total_row_groups = parquet_file.num_row_groups

# Se genera un bucle para leer los fragmentos del archivo.
for i in range(0, total_row_groups):
    # Se lee el fragmento del archivo.
    lote = parquet_file.read_row_group(i, use_pandas_metadata=True).to_pandas()

    # Se guarda el fragmento del archivo en un archivo parquet.
    lote.to_parquet(f"review_{i}.parquet", index=False)

### Rutas a los archivos seccionados.

In [5]:
path_0 ='C:\\Users\\fedez\\OneDrive\\Escritorio\\ProyectoG4-Google_Yelp\\Data\\Raw\\review_0.parquet'
path_1 ='C:\\Users\\fedez\\OneDrive\\Escritorio\\ProyectoG4-Google_Yelp\\Data\\Raw\\review_1.parquet'
path_2 ='C:\\Users\\fedez\\OneDrive\\Escritorio\\ProyectoG4-Google_Yelp\\Data\\Raw\\review_2.parquet'
path_3 ='C:\\Users\\fedez\\OneDrive\\Escritorio\\ProyectoG4-Google_Yelp\\Data\\Raw\\review_3.parquet'
path_4 ='C:\\Users\\fedez\\OneDrive\\Escritorio\\ProyectoG4-Google_Yelp\\Data\\Raw\\review_4.parquet'
path_5 ='C:\\Users\\fedez\\OneDrive\\Escritorio\\ProyectoG4-Google_Yelp\\Data\\Raw\\review_5.parquet'
path_6 ='C:\\Users\\fedez\\OneDrive\\Escritorio\\ProyectoG4-Google_Yelp\\Data\\Raw\\review_6.parquet'

### Valores nulos.

In [ ]:
def chequeo_de_nulos_parquet(file_path):
    try:
        # Leer el archivo Parquet en un DataFrame
        df = pd.read_parquet(file_path)
        
        # Verificar valores nulos
        null_counts = df.isnull().sum()
        total_nulls = null_counts.sum()
        
        # Imprimir resultados
        print(f'Archivo: {os.path.basename(file_path)}')
        print(f'Total de valores nulos: {total_nulls}')
        print('Valores nulos por columna:')
        print(null_counts)
        print('-' * 40)
    except Exception as e:
        print(f'Error al procesar el archivo {file_path}: {e}')

In [ ]:
chequeo_de_nulos_parquet(path_0)

Archivo: review_0.parquet
Total de valores nulos: 0
Valores nulos por columna:
review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64
----------------------------------------


In [ ]:
chequeo_de_nulos_parquet(path_1)

Archivo: review_1.parquet
Total de valores nulos: 0
Valores nulos por columna:
review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64
----------------------------------------


In [ ]:
chequeo_de_nulos_parquet(path_2)

Archivo: review_2.parquet
Total de valores nulos: 0
Valores nulos por columna:
review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64
----------------------------------------


In [ ]:
chequeo_de_nulos_parquet(path_3)

Archivo: review_3.parquet
Total de valores nulos: 0
Valores nulos por columna:
review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64
----------------------------------------


In [ ]:
chequeo_de_nulos_parquet(path_4)

Archivo: review_4.parquet
Total de valores nulos: 0
Valores nulos por columna:
review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64
----------------------------------------


In [ ]:
chequeo_de_nulos_parquet(path_5)

Archivo: review_5.parquet
Total de valores nulos: 0
Valores nulos por columna:
review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64
----------------------------------------


In [ ]:
chequeo_de_nulos_parquet(path_6)

Archivo: review_6.parquet
Total de valores nulos: 0
Valores nulos por columna:
review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64
----------------------------------------


Se concluye que no hay presencia de datos nulos en ninguno de los archivos fraccionados.

### Valores duplicados

In [ ]:
def chequeo_eliminación_duplicados(ruta):
    try:
        # Se lee el archivo Parquet en un DataFrame.
        df = pd.read_parquet(ruta)
        
        # Se verifica duplicados.
        duplicates = df.duplicated().sum()
        
        if duplicates > 0:
            print(f'Archivo: {os.path.basename(ruta)}')
            print(f'Total de filas duplicadas encontradas: {duplicates}')
            
            # Se eliminan duplicados.
            df = df.drop_duplicates()
            print(f'Filas duplicadas eliminadas. Total de filas restantes: {len(df)}')
            
            # Se guarda el DataFrame sin duplicados archivo parquet
            df.to_parquet(ruta, index=False)
            print(f'Archivo actualizado sin duplicados: {ruta}')
        else:
            print(f'Archivo: {os.path.basename(ruta)}')
            print('No se encontraron filas duplicadas.')
        
        print('-' * 40)
    except Exception as e:
        print(f'Error al procesar el archivo {ruta}: {e}')

In [ ]:
chequeo_eliminación_duplicados(path_0)

Archivo: review_0.parquet
No se encontraron filas duplicadas.
----------------------------------------


In [ ]:
chequeo_eliminación_duplicados(path_1)

Archivo: review_1.parquet
No se encontraron filas duplicadas.
----------------------------------------


In [ ]:
chequeo_eliminación_duplicados(path_2)

Archivo: review_2.parquet
No se encontraron filas duplicadas.
----------------------------------------


In [ ]:
chequeo_eliminación_duplicados(path_3)

Archivo: review_3.parquet
No se encontraron filas duplicadas.
----------------------------------------


In [ ]:
chequeo_eliminación_duplicados(path_4)

Archivo: review_4.parquet
No se encontraron filas duplicadas.
----------------------------------------


In [ ]:
chequeo_eliminación_duplicados(path_5)

Archivo: review_5.parquet
No se encontraron filas duplicadas.
----------------------------------------


In [ ]:
chequeo_eliminación_duplicados(path_6)

Archivo: review_6.parquet
No se encontraron filas duplicadas.
----------------------------------------


No se observan filas duplicadas en ninguno de los archivos fraccionados.

### Filtro de datos.

Se aplicará un filtro a cada fragmento para que se reduzca el número de filas a las necesarias.

In [6]:
df_0 = pd.read_parquet(path_0, engine='fastparquet')

In [58]:
(f'El dataframe presenta {len(df_0)} filas')

'El dataframe presenta 1048576 filas'

In [7]:
# Se filtran las filas que no contengan los "business_ids" correspondientes.
df_0_final = df_0[df_0['business_id'].isin(business_ids)]
# Se realiza el reset del índice.
df_0_final = df_0_final.reset_index(drop=True, inplace=False)

In [8]:
df_1 = pd.read_parquet(path_1, engine='fastparquet')

In [61]:
(f'El dataframe presenta {len(df_1)} filas')

'El dataframe presenta 1048576 filas'

In [9]:
# Se filtran las filas que no contengan los "business_ids" correspondientes.
df_1_final = df_1[df_1['business_id'].isin(business_ids)]
# Se realiza el reset del índice.
df_1_final = df_1_final.reset_index(drop=True, inplace=False)

In [10]:
df_2 = pd.read_parquet(path_2, engine='fastparquet')

In [64]:
(f'El dataframe presenta {len(df_2)} filas')

'El dataframe presenta 1048576 filas'

In [11]:
# Se filtran las filas que no contengan los "business_ids" correspondientes.
df_2_final = df_2[df_2['business_id'].isin(business_ids)]
# Se realiza el reset del índice.
df_2_final = df_2_final.reset_index(drop=True, inplace=False)

In [12]:
df_3 = pd.read_parquet(path_3, engine='fastparquet')

In [67]:
(f'El dataframe presenta {len(df_3)} filas')

'El dataframe presenta 1048576 filas'

In [13]:
# Se filtran las filas que no contengan los "business_ids" correspondientes.
df_3_final = df_3[df_3['business_id'].isin(business_ids)]
# Se realiza el reset del índice.
df_3_final = df_3_final.reset_index(drop=True, inplace=False)

In [14]:
df_4 = pd.read_parquet(path_4, engine='fastparquet')

In [70]:
(f'El dataframe presenta {len(df_4)} filas')

'El dataframe presenta 1048576 filas'

In [15]:
# Se filtran las filas que no contengan los "business_ids" correspondientes.
df_4_final = df_4[df_4['business_id'].isin(business_ids)]
# Se realiza el reset del índice.
df_4_final = df_4_final.reset_index(drop=True, inplace=False)

In [16]:
df_5 = pd.read_parquet(path_5, engine='fastparquet')

In [73]:
(f'El dataframe presenta {len(df_5)} filas')

'El dataframe presenta 1048576 filas'

In [17]:
# Se filtran las filas que no contengan los "business_ids" correspondientes.
df_5_final = df_5[df_5['business_id'].isin(business_ids)]
# Se realiza el reset del índice.
df_5_final = df_5_final.reset_index(drop=True, inplace=False)

In [18]:
df_6 = pd.read_parquet(path_6, engine='fastparquet')

In [76]:
(f'El dataframe presenta {len(df_6)} filas')

'El dataframe presenta 698824 filas'

In [19]:
# Se filtran las filas que no contengan los "business_ids" correspondientes.
df_6_final = df_6[df_6['business_id'].isin(business_ids)]
# Se realiza el reset del índice.
df_6_final = df_6_final.reset_index(drop=True, inplace=False)

Filas totales.

In [78]:
total_filas_raw = len(df_0) + len(df_1) + len(df_2) + len(df_3) + len(df_4) + len(df_5) + len(df_6)
print(f"El número de filas original es {total_filas_raw}")

El número de filas original es 6990280


### Concatenación de dataframes.

In [53]:
df_review = pd.concat([df_0_final, df_1_final, df_2_final, df_3_final, df_4_final, df_5_final, df_6_final], ignore_index=False)

In [80]:
df_review.head(3)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,TcCcHzc3L6Aboq3DteEfZA,OuatwND396ZQxm2zK8WlUQ,jNL5KUPz2-tHUJM__ysSaw,1,1,0,0,If you want to pay for everything a la carte t...,2014-08-24 20:14:12
1,cCs7yPSyk8NdA-Ufoz_7hw,FodM8aoGMQO2zsQCQxBTYQ,v5ktgWMAARaczTMh2rAJKg,2,0,0,0,The service here has gone down. We used to go ...,2017-12-27 02:57:57
2,eP-S5CuNNVdGsfmVc5BXpg,jDLeW4d-8WqMkikCTHFJ5g,kJpoduG3wdA35WzOF_T1Aw,5,0,0,0,Only the best Mexican food in Florida!!!! Sin...,2016-04-23 18:01:24


### Columna "stars".  
Verificación de outliers.

In [45]:
description = df_review['stars'].describe()
print(description)

count    72145.000000
mean         3.751584
std          1.454863
min          1.000000
25%          3.000000
50%          4.000000
75%          5.000000
max          5.000000
Name: stars, dtype: float64


In [54]:
df_review.rename(columns={'stars': 'stars review'}, inplace=True)
df_review.head(1)

,review_id,user_id,business_id,stars review,useful,funny,cool,text,date
0,TcCcHzc3L6Aboq3DteEfZA,OuatwND396ZQxm2zK8WlUQ,jNL5KUPz2-tHUJM__ysSaw,1,1,0,0,If you want to pay for everything a la carte t...,2014-08-24 20:14:12


Se puede apreciar que la columna "stars" no presenta outliers.

### Columna "date".

In [22]:
fecha_maxima = df_review['date'].max()
fecha_minima = df_review['date'].min()

print(f'La fecha mínima es {fecha_minima} y la fecha máxima es {fecha_maxima}')

La fecha mínima es 2006-02-02 19:09:03 y la fecha máxima es 2022-01-19 19:21:56


Para obtener un contexto en el marco más actualizado se procede a utilizar los datos que se encuentran dentro de un rango temporal entre 2017 y 2021. Por lo tanto, se procede a limitar las filas usadas a las que se encuentren en dicho rango temporal.

In [55]:
df_review = df_review[df_review['date'].between('2017-01-01', '2021-12-31', inclusive='both')]

Se corrobora que se hayan realizado los cambios correctamente.

In [56]:
fecha_maxima = df_review['date'].max()
fecha_minima = df_review['date'].min()

print(f'La fecha mínima es {fecha_minima} y la fecha máxima es {fecha_maxima}')

La fecha mínima es 2017-01-01 01:30:23 y la fecha máxima es 2021-12-30 23:36:35


Se crean columnas específicas para cada formato temporal.

In [57]:
df_review['date'] = pd.to_datetime(df_review['date'])

# Se crea una nueva columna con la fecha en formato 'YYYY-MM-DD'.
df_review['fecha review'] = df_review['date'].dt.date

# Se crea una nueva columna con la hora en formato de 24 horas enteras.
df_review['hora entera review'] = df_review['date'].dt.hour

df_review['dia review'] = df_review['date'].dt.strftime('%A')

# Se crea un diccionario para mapear los dias de la semana en español.
dias_semana_es = {
    'Monday': 'Lunes',
    'Tuesday': 'Martes',
    'Wednesday': 'Miércoles',
    'Thursday': 'Jueves',
    'Friday': 'Viernes',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}
df_review['dia review'] = df_review['dia review'].map(dias_semana_es)

# A extraerse los datos necesarios se elimina la columna "date".
df_review.drop(columns=['date'], inplace=True)

In [59]:
df_review.head(3)

,review_id,user_id,business_id,stars review,useful,funny,cool,text,fecha review,hora entera review,dia review
1,cCs7yPSyk8NdA-Ufoz_7hw,FodM8aoGMQO2zsQCQxBTYQ,v5ktgWMAARaczTMh2rAJKg,2,0,0,0,The service here has gone down. We used to go ...,2017-12-27,2,Miércoles
9,8ZDUB-Y6JFxex541jHprGg,bIXslp2Sojhp9OcA-PNTvw,-zsvmEbkd-K9K2DAAKqiEQ,5,0,0,0,Thoroughly enjoyed our visit to Salty's this e...,2018-07-17,2,Martes
11,Naht5FrBGZDFGjwCQ-DyMw,QKoP7XabOXXWIkTwvYyCiA,xGUAa3xa8KsmbolC3XboQg,5,0,0,0,Xtreme Tacos has become one of my favorite pla...,2017-10-02,16,Lunes


### Eliminación de columnas.  
Se opta por eliminar las columnas "useful", "funny" y "cool" ya que no aportan datos de valor para el análisis.

In [60]:
df_review = df_review.drop(columns=['useful', 'funny', 'cool'])

In [61]:
df_review_final = df_review.reset_index(drop=True, inplace=False)
df_review_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45615 entries, 0 to 45614
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   review_id           45615 non-null  object
 1   user_id             45615 non-null  object
 2   business_id         45615 non-null  object
 3   stars review        45615 non-null  int64 
 4   text                45615 non-null  object
 5   fecha review        45615 non-null  object
 6   hora entera review  45615 non-null  int32 
 7   dia review          45615 non-null  object
dtypes: int32(1), int64(1), object(6)
memory usage: 2.6+ MB


### Como se ha realizado una reducción de los datos a nivel temporal a través del dataframe "df_reviews" se usará los datos de la columna "business_id" para descartar las filas de los demás datasets que no coincidan con dicho ID.

In [90]:
# Se genera el filtro buisness_ids para continuar con la reducción de los dataframes.
business_ids = df_review_final['business_id'].unique()
print(f"El filtro consta de {len(business_ids)} ID's únicos")

El filtro consta de 628 ID's únicos


### También se genera un segundo filtro para los dataset que solo presentan una columna de "user_id" y se reduzcan al número de usuarios que hicieron reseñas dentro del marco del estado de Florida, en restaurantes mexicanos y durante el período entre 2017 y 2021.

In [12]:
# Se genera el filtro buisness_ids para continuar con la reducción de los dataframes.
user_ids = df_review_final['user_id'].unique()
print(f"El filtro consta de {len(user_ids)} ID's únicos")

El filtro consta de 31536 ID's únicos


## Archivo "checkin.json"

### Extracción.

In [ ]:
checkin = []

with open(path_checkin, 'r') as archivo:
    #Se crea un loop para ir incorporando los elementos a la lista.
    for linea in archivo:
        data= json.loads(linea)
        checkin.append(data)

In [ ]:
df_checkin = pd.DataFrame(checkin)
df_checkin.head()

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."
2,--30_8IhuyMHbSOcNWd6DQ,"2013-06-14 23:29:17, 2014-08-13 23:20:22"
3,--7PUidqRWpRSpXebiyxTg,"2011-02-15 17:12:00, 2011-07-28 02:46:10, 2012..."
4,--7jw19RH9JKXgFohspgQw,"2014-04-21 20:42:11, 2014-04-28 21:04:46, 2014..."


In [ ]:
df_checkin.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131930 entries, 0 to 131929
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   business_id  131930 non-null  object
 1   date         131930 non-null  object
dtypes: object(2)
memory usage: 2.0+ MB


### Valores nulos.

In [ ]:
df_checkin.isna().sum()

business_id    0
date           0
dtype: int64

Se observa que el archivo no presenta valores nulos en ninguna de sus columnas.

### Datos duplicados.

In [ ]:
# Se corrobora que no haya filas duplicadas en la columna "business_id".
duplicados = df_checkin['business_id'].duplicated()
duplicados_df = df_checkin[duplicados]
duplicados_df.shape

(0, 2)

No se observan datos duplicados en el dataset.

### Filtro de datos.

In [ ]:
# Se filtran las filas que no contengan los "business_ids" correspondientes.
df_checkin_final = df_checkin[df_checkin['business_id'].isin(business_ids)]
# Se realiza el reset del índice.
df_checkin_final = df_checkin_final.reset_index(drop=True, inplace=False)
df_checkin_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 616 entries, 0 to 615
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   business_id  616 non-null    object
 1   date         616 non-null    object
dtypes: object(2)
memory usage: 9.8+ KB


## Archivo "tip.json"

### Extracción.

In [ ]:
tip = [] 
#Se abre el archivo "tip.json".
with open(path_tip, encoding="utf-8") as archivo:
    # EL bucle irá incorporando los datos a la lista.
    for linea in archivo.readlines():
        tip.append(ast.literal_eval(linea))

#Se crea el dataframe "df_tip"
df_tip = pd.DataFrame(tip)

In [ ]:
df_tip.head()

,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0


In [ ]:
df_tip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 908915 entries, 0 to 908914
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   user_id           908915 non-null  object
 1   business_id       908915 non-null  object
 2   text              908915 non-null  object
 3   date              908915 non-null  object
 4   compliment_count  908915 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 34.7+ MB


### Valores nulos.

In [ ]:
df_tip.isna().sum()

user_id             0
business_id         0
text                0
date                0
compliment_count    0
dtype: int64

Se observa que el archivo no presenta valores nulos.

### Valores duplicados.

Se corrobora que si hay filas duplicadas en base al contenido de las columnas "user_id", "text" y "date"

In [ ]:
valores_duplicados = df_tip.duplicated(subset=['user_id', 'text', 'date'], keep=False)
filas_valores_duplicados = df_tip[valores_duplicados]
filas_valores_duplicados.shape

(126, 5)

Se eliminan las filas que poseen valores duplicados en las columnas "user_id", "text" y "date".

In [ ]:
df_tip = df_tip.drop_duplicates(subset=['user_id', 'text', 'date'], keep='first')

In [ ]:
# Se verifica que se hayan realizado los cambios.
valores_duplicados = df_tip.duplicated(subset=['user_id', 'text', 'date'], keep=False)
filas_valores_duplicados = df_tip[valores_duplicados]
filas_valores_duplicados.shape

(0, 5)

### Manejo de columnas.

### Columna "date".

In [ ]:
fecha_maxima = df_tip['date'].max()
fecha_minima = df_tip['date'].min()

print(f'La fecha mínima es {fecha_minima} y la fecha máxima es {fecha_maxima}')


La fecha mínima es 2009-04-16 13:11:49 y la fecha máxima es 2022-01-19 20:38:55


Se crean columnas específicas para cada formato temporal.

In [ ]:
df_tip['date'] = pd.to_datetime(df_tip['date'])

# Se crea una nueva columna con la fecha en formato 'YYYY-MM-DD'.
df_tip['fecha'] = df_tip['date'].dt.date

# Se crea una nueva columna con la hora en formato de 24 horas enteras.
df_tip['hora'] = df_tip['date'].dt.hour

df_tip['dia_semana'] = df_tip['date'].dt.strftime('%A')

# Se crea un diccionario para mapear los dias de la semana y traducirlos al español.
dias_semana_es = {
    'Monday': 'Lunes',
    'Tuesday': 'Martes',
    'Wednesday': 'Miércoles',
    'Thursday': 'Jueves',
    'Friday': 'Viernes',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}
df_tip['dia_semana'] = df_tip['dia_semana'].map(dias_semana_es)

In [ ]:
df_tip.head(1)

,user_id,business_id,text,date,compliment_count,fecha,hora,dia_semana
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0,2012-05-18,2,Viernes


### Columna "compliment_count".

Se oopta por eliminar la columna ya que no aporta datos relevantes.

In [ ]:
df_tip = df_tip.drop(['compliment_count'], axis=1)

In [ ]:
df_tip.head(1)

,user_id,business_id,text,date,fecha,hora,dia_semana
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,2012-05-18,2,Viernes


### Filtro de datos.

In [ ]:
# Se seleccionan los registros correspondientes al rango temporal entre 2017 y 2021.
df_tip_1 = df_tip[df_tip['date'].between('2017-01-01', '2021-12-31', inclusive='both')]
# Se filtran los registros que corresponden al "user_id" a utilizar.
df_tip_final = df_tip_1[df_tip_1['user_id'].isin(user_ids)]
# Se realiza el reset del índice.
df_tidf_tip_final =df_tip_final.reset_index(drop=True, inplace=False)
df_tidf_tip_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24009 entries, 0 to 24008
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   user_id      24009 non-null  object        
 1   business_id  24009 non-null  object        
 2   text         24009 non-null  object        
 3   date         24009 non-null  datetime64[ns]
 4   fecha        24009 non-null  object        
 5   hora         24009 non-null  int32         
 6   dia_semana   24009 non-null  object        
dtypes: datetime64[ns](1), int32(1), object(5)
memory usage: 1.2+ MB


## Archivo "user.parquet"

### Extracción.

In [4]:
df_user = pd.read_parquet(path_user, engine='fastparquet')

In [5]:
df_user.head(3)

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,3.91,250,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,3.74,1145,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,3.32,89,13,10,17,3,66,96,119,119,35,18


In [6]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2105597 entries, 0 to 2105596
Data columns (total 22 columns):
 #   Column              Dtype  
---  ------              -----  
 0   user_id             object 
 1   name                object 
 2   review_count        int64  
 3   yelping_since       object 
 4   useful              int64  
 5   funny               int64  
 6   cool                int64  
 7   elite               object 
 8   friends             object 
 9   fans                int64  
 10  average_stars       float64
 11  compliment_hot      int64  
 12  compliment_more     int64  
 13  compliment_profile  int64  
 14  compliment_cute     int64  
 15  compliment_list     int64  
 16  compliment_note     int64  
 17  compliment_plain    int64  
 18  compliment_cool     int64  
 19  compliment_funny    int64  
 20  compliment_writer   int64  
 21  compliment_photos   int64  
dtypes: float64(1), int64(16), object(5)
memory usage: 353.4+ MB


### Valores nulos.

In [7]:
df_user.isna().sum()

user_id               0
name                  0
review_count          0
yelping_since         0
useful                0
funny                 0
cool                  0
elite                 0
friends               0
fans                  0
average_stars         0
compliment_hot        0
compliment_more       0
compliment_profile    0
compliment_cute       0
compliment_list       0
compliment_note       0
compliment_plain      0
compliment_cool       0
compliment_funny      0
compliment_writer     0
compliment_photos     0
dtype: int64

### Valores duplicados.

In [8]:
duplicados = df_user['user_id'].duplicated()
duplicados_user = df_user[duplicados]
duplicados_user.shape

(117700, 22)

Se observa que en el dataset hay 117700 filas duplicadas las cuales serán eliminadas.

In [9]:
df_user = df_user.drop_duplicates(keep='first', inplace=False)

In [10]:
duplicados = df_user['user_id'].duplicated()
duplicados_user = df_user[duplicados]
duplicados_user.shape

(0, 22)

### Manejo de columnas.

### Columnas "review_count" y "useful".  
Verificación de outliers.

In [ ]:
columnas = ['review_count', 'useful']
descripcion = df_user[columnas].describe()
print(descripcion)

       review_count     useful
count    1987897.00 1987897.00
mean          23.39      42.30
std           82.57     641.48
min            0.00       0.00
25%            2.00       0.00
50%            5.00       3.00
75%           17.00      13.00
max        17473.00  206296.00


Se distingue que el dataset no presenta valores negativos en ninguna de las columnas, pero sí se muestran conteos de 17473 para "review_count" y 206296 para "useful"

In [13]:
# Se renombra la columna "review_count".
df_user.rename(columns={'review_count': 'user review count'}, inplace=True)
df_user.head(1)

,user_id,name,user review count,yelping_since,useful,funny,cool,elite,friends,fans,average_stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,3.91,250,65,55,56,18,232,844,467,467,239,180


### Columnas "average_stars".
Verificación de outliers.

In [ ]:
description = df_user['average_stars'].describe()
print(description)

count   1987897.00
mean          3.63
std           1.18
min           1.00
25%           3.00
50%           3.88
75%           4.56
max           5.00
Name: average_stars, dtype: float64


Se observa que la columna "average_stars" no presenta valores atípicos.

In [14]:
# Se renombra la columna.
df_user.rename(columns={'average_stars': 'avg user stars'}, inplace=True)
df_user.head(1)

,user_id,name,user review count,yelping_since,useful,funny,cool,elite,friends,fans,avg user stars,compliment_hot,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,3.91,250,65,55,56,18,232,844,467,467,239,180


Se decide eliminar las siguientes columnas ya que no aportan datos relevantes para el análisis.

In [15]:
df_user = df_user.drop(columns=['name','yelping_since','funny','cool','elite','friends','fans', 'compliment_hot', 'compliment_more','compliment_profile','compliment_cute','compliment_list','compliment_note',
                                'compliment_plain','compliment_cool','compliment_funny','compliment_writer','compliment_photos'])

In [16]:
df_user.head(3)

,user_id,user review count,useful,avg user stars
0,qVc8ODYU5SZjKXVBgXdI7w,585,7217,3.91
1,j14WgRoU_-2ZE1aw1dXrJg,4333,43091,3.74
2,2WnXYQFK0hXEoTxPtV2zvg,665,2086,3.32


### Filtro de datos.

In [17]:
# Se filtran las filas que no contengan los "business_ids" correspondientes.
df_user_final = df_user[df_user['user_id'].isin(user_ids)]
# Se realiza el reset del índice.
df_user_final = df_user_final.reset_index(drop=True, inplace=False)
df_user_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31536 entries, 0 to 31535
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   user_id            31536 non-null  object 
 1   user review count  31536 non-null  int64  
 2   useful             31536 non-null  int64  
 3   avg user stars     31536 non-null  float64
dtypes: float64(1), int64(2), object(1)
memory usage: 985.6+ KB


## Carga de datos.

Se crean archivos en formato parquet en base a los datasets que se fueron trabajando.

Archivo "business.parquet".

In [55]:
table = pa.Table.from_pandas(df_business_final)
pq.write_table(table, 'business.parquet')

Archivo "review.parquet".

In [62]:
table = pa.Table.from_pandas(df_review_final)
pq.write_table(table, 'review.parquet')

Archivo "checkin.parquet".

In [ ]:
table = pa.Table.from_pandas(df_checkin_final)
pq.write_table(table, 'checkin.parquet')

Archivo "tip.parquet".

In [ ]:
table = pa.Table.from_pandas(df_tip_final)
pq.write_table(table, 'tip.parquet')

Archivo "user.parquet"

In [18]:
table = pa.Table.from_pandas(df_user_final)
pq.write_table(table, 'user.parquet')

### Merge de dataframes.

Se crea una dataframe que combina "user", "business" y "review".

In [41]:
df_review_final = pd.read_parquet('C:\\Users\\fedez\\OneDrive\\Escritorio\\ProyectoG4-Google_Yelp\\Data\\Parquet\\review.parquet')
df_business_final = pd.read_parquet('C:\\Users\\fedez\\OneDrive\\Escritorio\\ProyectoG4-Google_Yelp\\Data\\Parquet\\business.parquet')
df_user_final = pd.read_parquet('C:\\Users\\fedez\\OneDrive\\Escritorio\\ProyectoG4-Google_Yelp\\Data\\Parquet\\user.parquet')

In [20]:
df_business_final.head(1)

,business_id,name,city,state,latitude,longitude,stars business,restaurant review count,categories,Lunes horarios,Martes horarios,Miércoles horarios,Jueves horarios,Viernes horarios,Sábado horarios,Domingo horarios
0,aNtKyc2rr-uK5cqzY9TVQQ,Chipotle Mexican Grill,Largo,Florida,27.894167,-82.779866,3.0,19,"Mexican, Fast Food, Restaurants",0:0-0:0,10:45-22:0,10:45-22:0,10:45-22:0,10:45-22:0,10:45-22:0,10:45-22:0


In [21]:
df_user_final.head(1)

,user_id,user review count,useful,avg user stars
0,rppTTi-kfF8-qyiArNemag,460,700,3.33


In [51]:
df_merge_1 =  pd.merge(df_review_final, df_user_final, on='user_id', how='left')
df_merge_final = pd.merge(df_business_final, df_merge_1, on='business_id', how='left')
df_merge_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72145 entries, 0 to 72144
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   business_id              72145 non-null  object        
 1   name                     72145 non-null  object        
 2   city                     72145 non-null  object        
 3   state                    72145 non-null  object        
 4   latitude                 72145 non-null  float64       
 5   longitude                72145 non-null  float64       
 6   stars business           72145 non-null  float64       
 7   restaurant review count  72145 non-null  int32         
 8   categories               72145 non-null  object        
 9   Lunes horarios           65280 non-null  object        
 10  Martes horarios          69194 non-null  object        
 11  Miércoles horarios       69318 non-null  object        
 12  Jueves horarios          69850 n

In [52]:
table = pa.Table.from_pandas(df_merge_final)
pq.write_table(table, 'Yelp!_final.parquet')